In [48]:
import struct
import numpy as np
import cv2
from IPython.display import display, Image, HTML
import glob

# =============================
#   描画パラメータ
# =============================
XMIN, XMAX = -60, 60
YMIN, YMAX = -60, 60
RES = 800  # 画像解像度

def load_frame(FNAME):
    with open(FNAME, "rb") as f:
        Nx = struct.unpack("q", f.read(8))[0]
        Ny = struct.unpack("q", f.read(8))[0]

        vx_all = np.zeros((Nx,Ny,4))
        vy_all = np.zeros((Nx,Ny,4))
        val_all = np.zeros((Nx,Ny))

        for i in range(Nx):
            for j in range(Ny):
                vx = struct.unpack("4d", f.read(32))
                vy = struct.unpack("4d", f.read(32))
                val = struct.unpack("d",  f.read(8))[0]

                vx_all[i,j] = vx
                vy_all[i,j] = vy
                val_all[i,j] = val

    return vx_all, vy_all, val_all


def world_to_img(x, y):
    ix = int((x - XMIN) / (XMAX - XMIN) * RES)
    iy = int((YMAX - y) / (YMAX - YMIN) * RES)
    return (ix, iy)


def frame_to_image(vx_all, vy_all, val_all):
    # normalize values
    vmin, vmax = val_all.min(), val_all.max()
    val_norm = (val_all - vmin) / (vmax - vmin + 1e-12)
    val_norm *= 255

    Nx, Ny = val_all.shape

    img = np.zeros((RES, RES, 3), dtype=np.uint8)

    for i in range(Nx):
        for j in range(Ny):
            pts = np.array([
                world_to_img(vx_all[i,j,k], vy_all[i,j,k])
                for k in range(4)
            ], np.int32)

            color = int(val_norm[i,j])
            cv2.fillPoly(img, [pts], (0,0,color))

    return img




In [ ]:
vx, vy, val = load_frame("../data/0D2V_cartesian/0.bin")
img = frame_to_image(vx, vy, val)
from IPython.display import Image, display
_, buf = cv2.imencode('.png', img)
display(Image(data=buf))


In [ ]:
# =============================
#   アニメーション作成
# =============================

files = sorted(glob.glob("../data/0D2V_cartesian/*.bin"), key=lambda s: int(s.split('/')[-1].split('.')[0]))
frames = []

for i, fn in enumerate(files):
    print(f"loading {fn} ({i+1}/{len(files)})")
    vx, vy, val = load_frame(fn)
    img = frame_to_image(vx, vy, val)
    frames.append(img)

# GIF 保存
gif_path = "../output/0d2v_cartesian.gif"
fps = 15
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
vid = cv2.VideoWriter("vlasov.mp4", fourcc, fps, (RES, RES))

for frame in frames:
    vid.write(frame)
vid.release()

# GIF生成（imageio必要: pip install imageio）
import imageio
imageio.mimsave(gif_path, [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames], fps=fps)

print("Saved:", gif_path, "and vlasov.mp4")

# Jupyter で GIF 表示
display(HTML(f"<img src='{gif_path}' width=400>"))
